In [24]:
import openai
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast
import tiktoken

API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01"

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-search-curie-query-001",
      "owner": "organization-owner",
      "id": "text-search-curie-query-001-billium-ds",
      "status": "succeeded",
      "created_at": 1680716120,
      "updated_at": 1680716120,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-search-curie-doc-001",
      "owner": "organization-owner",
      "id": "text-search-curie-doc-001-billium-ds",
      "status": "succeeded",
      "created_at": 1680716251,
      "updated_at": 1680716251,
      "object": "deployment"
    }
  ],
  "object": "list"
}


In [25]:
df = pd.read_csv("C:\\Users\\risaha\OneDrive - Microsoft\\Learning\\OpenAI\\Traffic_Violations.csv") # example: df = pd.read_csv("c:\\test\\bill_sum_data.csv")df
# read only top 300 rows in data frame
df = df.head(30)
df

C:\Users\risaha\AppData\Local\Temp\ipykernel_32300\3237501425.py:1: DtypeWarning: Columns (19,20,21,22,23,24,25,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("C:\\Users\\risaha\OneDrive - Microsoft\\Learning\\OpenAI\\Traffic_Violations.csv") # example: df = pd.read_csv("c:\\test\\bill_sum_data.csv")df


,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation
0,fbc324ab-bc8d-4743-ba23-7f9f370005e1,08/11/2019,20:02:00,MCP,"2nd District, Bethesda","LEAVING UNATTENDED VEH. W/O STOPPING ENGINE, L...",CORDELL ST @ NORFOLK AVE.,38.989743,-77.097770,No,...,21-1101(a),Transportation Article,False,BLACK,M,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.9897433333333, -77.09777)"
1,a6d904ec-d666-4bc3-8984-f37a4b31854d,08/12/2019,13:41:00,MCP,"2nd District, Bethesda",EXCEEDING POSTED MAXIMUM SPEED LIMIT: 85 MPH I...,NBI270 AT MIDDLEBROOK RD,39.174110,-77.246170,No,...,21-801.1,Transportation Article,False,WHITE,M,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.17411, -77.24617)"
2,54a64f6a-df28-4b65-a335-08883866aa46,08/12/2019,21:00:00,MCP,"5th District, Germantown",DRIVING VEH W/ TV-TYPE RECEIVING VIDEO EQUIP T...,MIDDLEBROOK AN 355,39.182015,-77.238221,No,...,21-1129,Transportation Article,False,BLACK,M,GAITHERSBURG,MD,MD,A - Marked Patrol,"(39.1820155, -77.2382213333333)"
3,cf5479b6-9bc7-4216-a7b2-99e57ae932af,08/12/2019,21:43:00,MCP,"5th District, Germantown",DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,GERMANTOWN RD AND ALE HOUSE,39.160508,-77.284023,No,...,13-401(h),Transportation Article,False,BLACK,M,GERMANTOWN,MD,MD,A - Marked Patrol,"(39.1605076666667, -77.284023)"
4,5601ca35-8ee7-4f8e-9208-d89cde96d469,08/12/2019,21:30:00,MCP,"2nd District, Bethesda",FAILURE OF LICENSEE TO NOTIFY ADMINISTRATION O...,EASTWEST/ 355,38.984247,-77.090548,No,...,16-116(a),Transportation Article,False,BLACK,M,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.9842466666667, -77.0905483333333)"
5,5e641929-62f8-4313-a8a7-d4d8d72f3f2e,08/12/2019,22:32:00,MCP,"2nd District, Bethesda",DRIVING MOTOR VEHICLE ON HIGHWAY WITHOUT REQUI...,CONNECTICUT/ MONTROSE DRWY,39.001568,-77.076912,No,...,16-101(a1),Transportation Article,False,HISPANIC,M,SILVER SPRING,MD,XX,A - Marked Patrol,"(39.0015683333333, -77.0769116666667)"
6,5e641929-62f8-4313-a8a7-d4d8d72f3f2e,08/12/2019,22:32:00,MCP,"2nd District, Bethesda",FAILURE OF INDIVIDUAL DRIVING ON HIGHWAY TO DI...,CONNECTICUT/ MONTROSE DRWY,39.001568,-77.076912,No,...,16-112(c),Transportation Article,False,HISPANIC,M,SILVER SPRING,MD,XX,A - Marked Patrol,"(39.0015683333333, -77.0769116666667)"
7,5e641929-62f8-4313-a8a7-d4d8d72f3f2e,08/12/2019,22:32:00,MCP,"2nd District, Bethesda",FAILURE OF INDIVIDUAL DRIVING ON HWY. TO HAVE ...,CONNECTICUT/ MONTROSE DRWY,39.001568,-77.076912,No,...,16-112(b),Transportation Article,False,HISPANIC,M,SILVER SPRING,MD,XX,A - Marked Patrol,"(39.0015683333333, -77.0769116666667)"
8,5e641929-62f8-4313-a8a7-d4d8d72f3f2e,08/12/2019,22:32:00,MCP,"2nd District, Bethesda",FAILURE TO DISPLAY REGISTRATION CARD UPON DEMA...,CONNECTICUT/ MONTROSE DRWY,39.001568,-77.076912,No,...,13-409(b),Transportation Article,False,HISPANIC,M,SILVER SPRING,MD,XX,A - Marked Patrol,"(39.0015683333333, -77.0769116666667)"
9,5e641929-62f8-4313-a8a7-d4d8d72f3f2e,08/12/2019,22:32:00,MCP,"2nd District, Bethesda",FAILURE TO CARRY REGISTRATION CARD WITHIN VEHICLE,CONNECTICUT/ MONTROSE DRWY,39.001568,-77.076912,No,...,13-409(a),Transportation Article,False,HISPANIC,M,SILVER SPRING,MD,XX,A - Marked Patrol,"(39.0015683333333, -77.0769116666667)"


In [26]:
df_violation = df[['Description', 'Accident', 'Charge','Contributed To Accident','Race']]
df_violation

,Description,Accident,Charge,Contributed To Accident,Race
0,"LEAVING UNATTENDED VEH. W/O STOPPING ENGINE, L...",No,21-1101(a),False,BLACK
1,EXCEEDING POSTED MAXIMUM SPEED LIMIT: 85 MPH I...,No,21-801.1,False,WHITE
2,DRIVING VEH W/ TV-TYPE RECEIVING VIDEO EQUIP T...,No,21-1129,False,BLACK
3,DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,No,13-401(h),False,BLACK
4,FAILURE OF LICENSEE TO NOTIFY ADMINISTRATION O...,No,16-116(a),False,BLACK
5,DRIVING MOTOR VEHICLE ON HIGHWAY WITHOUT REQUI...,No,16-101(a1),False,HISPANIC
6,FAILURE OF INDIVIDUAL DRIVING ON HIGHWAY TO DI...,No,16-112(c),False,HISPANIC
7,FAILURE OF INDIVIDUAL DRIVING ON HWY. TO HAVE ...,No,16-112(b),False,HISPANIC
8,FAILURE TO DISPLAY REGISTRATION CARD UPON DEMA...,No,13-409(b),False,HISPANIC
9,FAILURE TO CARRY REGISTRATION CARD WITHIN VEHICLE,No,13-409(a),False,HISPANIC


In [27]:
# s is input text
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    # remove all instances of multiple spaces
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df_violation['Description'] = df_violation["Description"].apply(lambda x : normalize_text(x))
df_violation

C:\Users\risaha\AppData\Local\Temp\ipykernel_32300\2693486740.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_violation['Description'] = df_violation["Description"].apply(lambda x : normalize_text(x))


,Description,Accident,Charge,Contributed To Accident,Race
0,"LEAVING UNATTENDED VEH. W/O STOPPING ENGINE, L...",No,21-1101(a),False,BLACK
1,EXCEEDING POSTED MAXIMUM SPEED LIMIT: 85 MPH I...,No,21-801.1,False,WHITE
2,DRIVING VEH W/ TV-TYPE RECEIVING VIDEO EQUIP T...,No,21-1129,False,BLACK
3,DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,No,13-401(h),False,BLACK
4,FAILURE OF LICENSEE TO NOTIFY ADMINISTRATION O...,No,16-116(a),False,BLACK
5,DRIVING MOTOR VEHICLE ON HIGHWAY WITHOUT REQUI...,No,16-101(a1),False,HISPANIC
6,FAILURE OF INDIVIDUAL DRIVING ON HIGHWAY TO DI...,No,16-112(c),False,HISPANIC
7,FAILURE OF INDIVIDUAL DRIVING ON HWY. TO HAVE ...,No,16-112(b),False,HISPANIC
8,FAILURE TO DISPLAY REGISTRATION CARD UPON DEMA...,No,13-409(b),False,HISPANIC
9,FAILURE TO CARRY REGISTRATION CARD WITHIN VEHICLE,No,13-409(a),False,HISPANIC


In [28]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [35]:
# tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# n_token = tokenizer.tokenize("EXCEEDING POSTED MAXIMUM SPEED LIMIT: 85 MPH I. N. T. 85 MPH")
# len(n_token)
# # df_violation['n_tokens'] = df_violation["Description"].apply(lambda x: len(tokenizer.encode(x)))
# # df_violation = df_violation[df_violation.n_tokens<2000]
# # len(df_violation)



top_n = 1000
#df_violation = df_violation.sort_values("Time").tail(top_n * 2)  # first cut to first 2k entries, assuming less than half will be filtered out
#df.drop("Time", axis=1, inplace=True)

encoding = tiktoken.get_encoding(embedding_encoding)

# omit reviews that are too long to embed
df_violation["n_tokens"] = df_violation["Description"].apply(lambda x: len(encoding.encode(x)))
df_violation = df_violation[df_violation.n_tokens <= max_tokens].tail(top_n)
len(df_violation)

30

In [36]:
df_violation

,Description,Accident,Charge,Contributed To Accident,Race,n_tokens
0,"LEAVING UNATTENDED VEH. W/O STOPPING ENGINE, L...",No,21-1101(a),False,BLACK,28
1,EXCEEDING POSTED MAXIMUM SPEED LIMIT: 85 MPH I...,No,21-801.1,False,WHITE,22
2,DRIVING VEH W/ TV-TYPE RECEIVING VIDEO EQUIP T...,No,21-1129,False,BLACK,27
3,DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,No,13-401(h),False,BLACK,13
4,FAILURE OF LICENSEE TO NOTIFY ADMINISTRATION O...,No,16-116(a),False,BLACK,17
5,DRIVING MOTOR VEHICLE ON HIGHWAY WITHOUT REQUI...,No,16-101(a1),False,HISPANIC,14
6,FAILURE OF INDIVIDUAL DRIVING ON HIGHWAY TO DI...,No,16-112(c),False,HISPANIC,23
7,FAILURE OF INDIVIDUAL DRIVING ON HWY. TO HAVE ...,No,16-112(b),False,HISPANIC,18
8,FAILURE TO DISPLAY REGISTRATION CARD UPON DEMA...,No,13-409(b),False,HISPANIC,16
9,FAILURE TO CARRY REGISTRATION CARD WITHIN VEHICLE,No,13-409(a),False,HISPANIC,11


In [9]:
understand_tokenization = tokenizer.tokenize(df_violation.Description[0])
understand_tokenization

['S',
 'ECTION',
 'Ġ1',
 '.',
 'ĠSH',
 'ORT',
 'ĠTIT',
 'LE',
 '.',
 'ĠThis',
 'ĠAct',
 'Ġmay',
 'Ġbe',
 'Ġcited',
 'Ġas',
 'Ġthe',
 'Ġ``',
 'National',
 'ĠScience',
 'ĠEducation',
 'ĠTax',
 'ĠIn',
 'cent',
 'ive',
 'Ġfor',
 'ĠBusiness',
 'es',
 'ĠAct',
 'Ġof',
 'Ġ2007',
 "''.",
 'ĠSEC',
 '.',
 'Ġ2',
 '.',
 'ĠCR',
 'EDIT',
 'S',
 'ĠFOR',
 'ĠC',
 'ER',
 'TAIN',
 'ĠCONTR',
 'IB',
 'UT',
 'IONS',
 'ĠBEN',
 'EF',
 'IT',
 'ING',
 'ĠSC',
 'IENCE',
 ',',
 'ĠTECH',
 'N',
 'OLOGY',
 ',',
 'ĠENG',
 'INE',
 'ER',
 'ING',
 ',',
 'ĠAND',
 'ĠM',
 'ATH',
 'EM',
 'AT',
 'ICS',
 'ĠED',
 'UC',
 'ATION',
 'ĠAT',
 'ĠTHE',
 'ĠELE',
 'MENT',
 'ARY',
 'ĠAND',
 'ĠSEC',
 'OND',
 'ARY',
 'ĠSCHOOL',
 'ĠLEVEL',
 '.',
 'Ġ(',
 'a',
 ')',
 'ĠIn',
 'ĠGeneral',
 '.--',
 'Sub',
 'part',
 'ĠD',
 'Ġof',
 'Ġpart',
 'ĠIV',
 'Ġof',
 'Ġsub',
 'chapter',
 'ĠA',
 'Ġof',
 'Ġchapter',
 'Ġ1',
 'Ġof',
 'Ġthe',
 'ĠInternal',
 'ĠRevenue',
 'ĠCode',
 'Ġof',
 'Ġ1986',
 'Ġ(',
 'rel',
 'ating',
 'Ġto',
 'Ġbusiness',
 'Ġrelated',
 'Ġcred

In [10]:
len(understand_tokenization)

1480

In [37]:
df_violation[embedding_model] = df_violation["Description"].apply(lambda x : get_embedding(x, engine = embedding_model))

In [38]:
df_violation

,Description,Accident,Charge,Contributed To Accident,Race,n_tokens,text-embedding-ada-002
0,"LEAVING UNATTENDED VEH. W/O STOPPING ENGINE, L...",No,21-1101(a),False,BLACK,28,"[-0.022329259663820267, -0.007283020298928022,..."
1,EXCEEDING POSTED MAXIMUM SPEED LIMIT: 85 MPH I...,No,21-801.1,False,WHITE,22,"[-0.008199581876397133, 0.013533543795347214, ..."
2,DRIVING VEH W/ TV-TYPE RECEIVING VIDEO EQUIP T...,No,21-1129,False,BLACK,27,"[-0.005379942711442709, -0.00332684931345284, ..."
3,DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,No,13-401(h),False,BLACK,13,"[-0.003529394743964076, 0.004659191705286503, ..."
4,FAILURE OF LICENSEE TO NOTIFY ADMINISTRATION O...,No,16-116(a),False,BLACK,17,"[-0.012890083715319633, -0.004125864710658789,..."
5,DRIVING MOTOR VEHICLE ON HIGHWAY WITHOUT REQUI...,No,16-101(a1),False,HISPANIC,14,"[-0.004720169585198164, 0.002585550071671605, ..."
6,FAILURE OF INDIVIDUAL DRIVING ON HIGHWAY TO DI...,No,16-112(c),False,HISPANIC,23,"[-0.008074385114014149, -7.155157800298184e-05..."
7,FAILURE OF INDIVIDUAL DRIVING ON HWY. TO HAVE ...,No,16-112(b),False,HISPANIC,18,"[-0.013304694555699825, 0.021044477820396423, ..."
8,FAILURE TO DISPLAY REGISTRATION CARD UPON DEMA...,No,13-409(b),False,HISPANIC,16,"[-0.017452102154493332, -0.0031961314380168915..."
9,FAILURE TO CARRY REGISTRATION CARD WITHIN VEHICLE,No,13-409(a),False,HISPANIC,11,"[-0.018658334389328957, 0.009770901873707771, ..."


In [39]:
# search through the reviews for a specific product
def search_docs(df, user_query, top_n=3, to_print=True):
    embedding = get_embedding(
        user_query,
        engine=embedding_model
    )
    df["similarities"] = df["text-embedding-ada-002"].apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    if to_print:
        display(res)
    return res


res = search_docs(df_violation, "what is the charge for leaving vehicle unattended", top_n=4)

,Description,Accident,Charge,Contributed To Accident,Race,n_tokens,text-embedding-ada-002,similarities
0,"LEAVING UNATTENDED VEH. W/O STOPPING ENGINE, L...",No,21-1101(a),False,BLACK,28,"[-0.022329259663820267, -0.007283020298928022,...",0.834720
9,FAILURE TO CARRY REGISTRATION CARD WITHIN VEHICLE,No,13-409(a),False,HISPANIC,11,"[-0.018658334389328957, 0.009770901873707771, ...",0.813112
3,DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,No,13-401(h),False,BLACK,13,"[-0.003529394743964076, 0.004659191705286503, ...",0.789410
5,DRIVING MOTOR VEHICLE ON HIGHWAY WITHOUT REQUI...,No,16-101(a1),False,HISPANIC,14,"[-0.004720169585198164, 0.002585550071671605, ...",0.787867


In [40]:
res["Charge"][0]

'21-1101(a)'